In [1]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoConfig
import tensorflow as tf
import numpy as np

# 모델 이름 설정
model_name = "hun3359/klue-bert-base-sentiment"

# 모델, 토크나이저, 설정 로드
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# 라벨 매핑 확인
labels = [config.id2label[i] for i in range(len(config.id2label))]
print("Labels:", labels)

# ✅ 1단계: 원본 모델 테스트
def test_original_model(text):
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=128)
    logits = model(**inputs).logits
    probs = tf.nn.softmax(logits, axis=-1).numpy()[0]
    return logits.numpy()[0], probs

# ✅ 2단계: 개선된 ConcreteFunction 정의
@tf.function
def model_inference(input_ids, attention_mask, token_type_ids=None):
    # token_type_ids도 포함하여 더 안정적인 추론
    if token_type_ids is not None:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    else:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    return outputs.logits

# ✅ 3단계: 대표 데이터셋 생성 (양자화 개선)
def representative_dataset():
    """대표 데이터셋 생성으로 양자화 품질 향상"""
    sample_texts = [
        "오늘은 너무 행복한 하루야!",
        "너무 피곤하고 지쳐.",
        "오늘은 정말 뿌듯했어",
        "평범한 하루였어",
        "별로 좋지도 나쁘지도 않아",
        "정말 최악의 하루였어",
        "기분이 좋네요",
        "화가 나는 일이 있었어"
    ]

    for text in sample_texts:
        inputs = tokenizer(text, return_tensors="tf", padding="max_length",
                          truncation=True, max_length=128)
        # 올바른 데이터 타입으로 캐스팅 (int32 유지)
        yield [
            inputs['input_ids'],
            inputs['attention_mask'],
            inputs.get('token_type_ids', tf.zeros_like(inputs['input_ids']))
        ]

# ✅ 4단계: 개선된 TFLite 변환 (여러 시나리오)
def convert_to_tflite_improved():
    print("시나리오 1: 양자화 없이 변환 시도...")
    try:
        # 구체적인 함수 생성
        concrete_func = model_inference.get_concrete_function(
            input_ids=tf.TensorSpec([1, 128], tf.int32, name="input_ids"),
            attention_mask=tf.TensorSpec([1, 128], tf.int32, name="attention_mask"),
            token_type_ids=tf.TensorSpec([1, 128], tf.int32, name="token_type_ids")
        )

        # TFLite 변환기 설정 (양자화 없음)
        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter.allow_custom_ops = True

        tflite_model = converter.convert()
        print("✅ 양자화 없는 변환 성공!")
        return tflite_model

    except Exception as e:
        print(f"❌ 양자화 없는 변환 실패: {e}")

    print("\n시나리오 2: 보수적 양자화로 변환 시도...")
    try:
        # 구체적인 함수 생성
        concrete_func = model_inference.get_concrete_function(
            input_ids=tf.TensorSpec([1, 128], tf.int32, name="input_ids"),
            attention_mask=tf.TensorSpec([1, 128], tf.int32, name="attention_mask"),
            token_type_ids=tf.TensorSpec([1, 128], tf.int32, name="token_type_ids")
        )

        # TFLite 변환기 설정
        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

        # 보수적 최적화 설정
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset

        # 타입 설정
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter.allow_custom_ops = True

        # 양자화 타겟 설정 (가중치만 양자화)
        converter.target_spec.supported_types = [tf.float16]

        tflite_model = converter.convert()
        print("✅ 보수적 양자화 변환 성공!")
        return tflite_model

    except Exception as e:
        print(f"❌ 보수적 양자화 변환 실패: {e}")

    print("\n시나리오 3: 기본 변환만 시도...")
    try:
        # 가장 기본적인 변환
        concrete_func = model_inference.get_concrete_function(
            input_ids=tf.TensorSpec([1, 128], tf.int32, name="input_ids"),
            attention_mask=tf.TensorSpec([1, 128], tf.int32, name="attention_mask"),
            token_type_ids=tf.TensorSpec([1, 128], tf.int32, name="token_type_ids")
        )

        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
        converter.allow_custom_ops = True

        tflite_model = converter.convert()
        print("✅ 기본 변환 성공!")
        return tflite_model

    except Exception as e:
        print(f"❌ 기본 변환도 실패: {e}")
        return None

# ✅ 5단계: TFLite 모델 테스트
def test_tflite_model(tflite_model, text):
    """TFLite 모델 테스트"""
    # 인터프리터 생성
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    # 입력/출력 세부사항 가져오기
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("Input details:", input_details)
    print("Output details:", output_details)

    # 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="np", padding="max_length",
                      truncation=True, max_length=128)

    # 입력 설정
    for i, detail in enumerate(input_details):
        if 'input_ids' in detail['name']:
            interpreter.set_tensor(detail['index'], inputs['input_ids'].astype(np.int32))
        elif 'attention_mask' in detail['name']:
            interpreter.set_tensor(detail['index'], inputs['attention_mask'].astype(np.int32))
        elif 'token_type_ids' in detail['name']:
            token_type_ids = inputs.get('token_type_ids', np.zeros_like(inputs['input_ids']))
            interpreter.set_tensor(detail['index'], token_type_ids.astype(np.int32))

    # 추론 실행
    interpreter.invoke()

    # 결과 가져오기
    logits = interpreter.get_tensor(output_details[0]['index'])
    probs = tf.nn.softmax(logits, axis=-1).numpy()[0]

    return logits[0], probs

# ✅ 6단계: 전체 테스트 실행
def run_comparison_test():
    test_texts = [
        "오늘은 너무 행복한 하루야!",
        "너무 피곤하고 지쳐.",
        "오늘은 정말 뿌듯했어",
        "평범한 하루였어"
    ]

    print("TFLite 모델 변환 중...")
    tflite_model = convert_to_tflite_improved()

    if tflite_model is None:
        print("TFLite 변환 실패!")
        return

    print("모델 변환 완료! 비교 테스트 시작...")
    print("="*60)

    for text in test_texts:
        print(f"\n테스트 텍스트: '{text}'")

        # 원본 모델 결과
        orig_logits, orig_probs = test_original_model(text)
        orig_pred = labels[np.argmax(orig_probs)]
        print(f"원본 모델: {orig_pred} ({orig_probs.max():.3f})")

        # TFLite 모델 결과
        try:
            tflite_logits, tflite_probs = test_tflite_model(tflite_model, text)
            tflite_pred = labels[np.argmax(tflite_probs)]
            print(f"TFLite 모델: {tflite_pred} ({tflite_probs.max():.3f})")

            # 차이 분석
            logits_diff = np.abs(orig_logits - tflite_logits).mean()
            probs_diff = np.abs(orig_probs - tflite_probs).mean()
            print(f"로짓 차이: {logits_diff:.4f}, 확률 차이: {probs_diff:.4f}")

        except Exception as e:
            print(f"TFLite 테스트 실패: {e}")

    # 모델 저장
    tflite_path = "sentiment_model_improved.tflite"
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print(f"\n개선된 모델 저장됨: {tflite_path}")

# 실행
if __name__ == "__main__":
    run_comparison_test()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.67k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Labels: ['분노', '툴툴대는', '좌절한', '짜증내는', '방어적인', '악의적인', '안달하는', '구역질 나는', '노여워하는', '성가신', '슬픔', '실망한', '비통한', '후회되는', '우울한', '마비된', '염세적인', '눈물이 나는', '낙담한', '환멸을 느끼는', '불안', '두려운', '스트레스 받는', '취약한', '혼란스러운', '당혹스러운', '회의적인', '걱정스러운', '조심스러운', '초조한', '상처', '질투하는', '배신당한', '고립된', '충격 받은', '가난한 불우한', '희생된', '억울한', '괴로워하는', '버려진', '당황', '고립된(당황한)', '남의 시선을 의식하는', '외로운', '열등감', '죄책감의', '부끄러운', '혐오스러운', '한심한', '혼란스러운(당황한)', '기쁨', '감사하는', '신뢰하는', '편안한', '만족스러운', '흥분', '느긋', '안도', '신이 난', '자신하는']
TFLite 모델 변환 중...
시나리오 1: 양자화 없이 변환 시도...


✅ 양자화 없는 변환 성공!
모델 변환 완료! 비교 테스트 시작...

테스트 텍스트: '오늘은 너무 행복한 하루야!'
원본 모델: 기쁨 (0.356)
Input details: [{'name': 'input_ids', 'index': 0, 'shape': array([  1, 128], dtype=int32), 'shape_signature': array([  1, 128], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'attention_mask', 'index': 1, 'shape': array([  1, 128], dtype=int32), 'shape_signature': array([  1, 128], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'token_type_ids', 'index': 2, 'shape': array([  1, 128], dtype=int32), 'shape_signature': array([  1, 128], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_paramet